In [4]:
# 패키지
import requests
from bs4 import BeautifulSoup
import time
import urllib.request #
from selenium.webdriver import Chrome
import re     
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
import datetime as dt
import pandas as pd
import konlpy

In [5]:
#크롬 드라이버 연결
delay=3
browser = Chrome()
browser.implicitly_wait(delay)

start_url  = 'https://www.youtube.com/channel/UCtckgmUcpzqGnzcs7xEqMzQ/videos'
browser.get(start_url)  
browser.maximize_window()

title_list = [] #제목 리스트 생성
href_list = [] #주소 리스트 생성
viewcountmake_list = [] #조회수 크롤링 위한 리스트 생성
viewcount_list = [] #조회수 리스트 생성

body = browser.find_element_by_tag_name('body')#스크롤하기 위해 소스 추출
num_of_pagedowns = 20

In [6]:
#스크롤 다운
while num_of_pagedowns:
    body.send_keys(Keys.PAGE_DOWN)
    time.sleep(2)
    num_of_pagedowns -= 1

# 페이지 소스 받아오기
html0 = browser.page_source
html = BeautifulSoup(html0,'html.parser')

# 구독자 수 저장
subsc = html.find(id="subscriber-count").text

# title 저장
for tr in html.find_all(id="video-title"):
    title = tr.get('title')
    title_list.append(title)

#href 태그 내용 저장
for tr in html.find_all(id="video-title"):
    href = tr.get('href')
    href_list.append(href)

# 조회수 저장
for tr in html.find_all('span', class_="style-scope ytd-grid-video-renderer"):
    viewcount = tr.get_text('span')
    viewcountmake_list.append(viewcount)

# 데이터 클리닝 작업(조회수와 등록일에서 등록일 제거)
for tr in range(0,len(viewcountmake_list),2):
    a = viewcountmake_list[tr]
    viewcount_list.append(a)

contents_count = len(title_list)

In [61]:
#구독자수, 동영상 수
peng_totlist = [subsc,contents_count]

#title, 주소, 좋아요수 데이터프레임 생성
peng_list = pd.DataFrame({'title':title_list, 'href':href_list, 'viewcount':clean_viewcount})

print(peng_totlist)
print(peng_list)

['구독자 130만명', 132]
                                            title                  href  \
0         영화 '천문: 하늘에 묻는다' 오디션 보러 간 펭수(*쿠키영상 있음!)  /watch?v=8qYmLVOlYXI   
1                         [Ep.72] 수업 중에 경운기를 몰아봤다  /watch?v=OD-Y-CxHz4A   
2                          [Ep.71] 내 헤드셋 누가 가져갔어?  /watch?v=rRZtVrh6kgI   
3                            펭수로 세계 대통합은 이미 시작됐다.  /watch?v=RKi98URQ6GM   
4                      [EP.70] 별주부전아닙니다 펭.수.전 입니다  /watch?v=mXUX4V_CYWw   
..                                            ...                   ...   
127  "2m 10cm 전학생  VS 교장선생님?" 펭수, 교장실 급습! [EP.02]  /watch?v=2gg1QKk7fjw   
128          "관종 펭귄, 초등학교 습격?" 펭수, 학교 가다! [EP.01]  /watch?v=M-52pdhp-ao   
129   관종 펭귄 '펭수' EBS 데뷔 D-3!! teaser.2 [자이언트 펭TV]  /watch?v=ErO16Y9sE8Q   
130    'EBS 연습생 펭귄, 방송국 점거??' [자이언트 펭TV] teaser.1  /watch?v=4vkzT47_Vdg   
131              [펭TV] EBS 연습생 펭수 유튜브 데뷔💜 머랭쿠키 먹방  /watch?v=XeH_8w4AWS4   

     viewcount  
0       950000  
1       550000  
2      1420000  
3       6400

In [12]:
# 조회수 데이터 전처리
clean_viewcount = []
for i in viewcount_list:
    a = i[4:-2]
    amul = float(a)*10000
    clean_viewcount.append(int(amul))
print(clean_viewcount)


[950000, 550000, 1420000, 640000, 1100000, 1330000, 880000, 1280000, 990000, 1360000, 1530000, 590000, 1900000, 640000, 1370000, 1360000, 870000, 730000, 1030000, 670000, 320000, 1410000, 580000, 890000, 790000, 700000, 410000, 2170000, 1160000, 180000, 1500000, 200000, 500000, 320000, 780000, 530000, 790000, 420000, 1710000, 470000, 540000, 550000, 350000, 3080000, 300000, 220000, 650000, 330000, 460000, 300000, 520000, 1320000, 99000, 2060000, 1490000, 370000, 820000, 1170000, 730000, 610000, 590000, 350000, 270000, 860000, 490000, 420000, 320000, 610000, 1330000, 290000, 290000, 320000, 90000, 1060000, 360000, 430000, 220000, 1500000, 160000, 290000, 560000, 180000, 220000, 95000, 780000, 1470000, 380000, 810000, 710000, 650000, 97000, 490000, 800000, 350000, 220000, 440000, 460000, 500000, 140000, 120000, 200000, 640000, 200000, 530000, 190000, 100000, 810000, 900000, 1010000, 900000, 220000, 490000, 660000, 670000, 470000, 900000, 220000, 380000, 320000, 460000, 530000, 410000, 28

In [19]:
# 영상별 본문내용
story_sub_list = [] # 영상 본문내용 리스트
start_url  = 'https://www.youtube.com/'
for i in href_list:
    url = start_url + i
    browser.get(url)  
    #browser.maximize_window()
    source = browser.page_source
    bs = BeautifulSoup(source, "html.parser")
    story = browser.find_element_by_id('content')
    story_source = browser.page_source
    bsstory = BeautifulSoup(story_source, "html.parser")
    print(i)
    # 콘텐츠 내용 설명
    story_sub = bsstory.find('yt-formatted-string', class_='content style-scope ytd-video-secondary-info-renderer').text
    story_sub_list.append(story_sub)
    
print(story_sub_list)

/watch?v=8qYmLVOlYXI
/watch?v=OD-Y-CxHz4A
/watch?v=rRZtVrh6kgI
/watch?v=RKi98URQ6GM
/watch?v=mXUX4V_CYWw
/watch?v=Ofi8fGaP6nU
/watch?v=dIsUJfd440o
/watch?v=qBkN_uIDjSc
/watch?v=13nTdgAZMAw
/watch?v=yUJAvW2Rykc
/watch?v=XUM3sH1kBtw
/watch?v=LPmyxMH96S8
/watch?v=wedLGh2jxkQ
/watch?v=25RhzK3HuYM
/watch?v=E5f4pRIycsU
/watch?v=2biW2UZBvVA
/watch?v=R2Oro64h5aM
/watch?v=iJF3R-5V2pU
/watch?v=Ja6O9CSzwuk
/watch?v=ThzyHA0LHvI
/watch?v=SlDD1ipgqjs
/watch?v=ze1wbV20qAU
/watch?v=1QNlKu67cwo
/watch?v=5fqCbkIvJuE
/watch?v=sxVKJXb3cp0
/watch?v=5OFDUu8UOvY
/watch?v=KTk8Bu4DRHk
/watch?v=UD-WQvgjsng
/watch?v=Qj2DYNju4Qo
/watch?v=tz_-lCADsbw
/watch?v=pv84Qcu9DOQ
/watch?v=ri5XpIgmoG8
/watch?v=A3_UpfZ1-pE
/watch?v=V4Jh4lNDFfk
/watch?v=AkNQlVM4DrU
/watch?v=WpfexQQkmP4
/watch?v=7SKjzUC77xg
/watch?v=yj_lTuH9qds
/watch?v=yN7wrzXtIWM
/watch?v=33nLNnZrBhE
/watch?v=vlqaa_YxxBE
/watch?v=8eIsC_9hXy0
/watch?v=JgfXjuxnrYM
/watch?v=K_5lal40lCk
/watch?v=zWGdwGfzgnY
/watch?v=I2nxlDdbY0U
/watch?v=S9RTFPo58tI
/watch?v=LwSx

In [20]:
# 본문내용 데이터 전처리
clean_story_sub_list = []
for i in story_sub_list:
    a = i.replace('\n', '')
    # 첨부 음악 리스트가 본문내용에 포함된 경우 찾아서 삭제
    delresult = a.find("[MUSIC INFO]")
    delresult1 = a.find("음원 정보")
    if delresult != -1:
        a = a[:delresult]
        clean_story_sub_list.append(a)
    elif delresult1 != -1:
        a = a[:delresult1]
        clean_story_sub_list.append(a)
    else:
        clean_story_sub_list.append(a)
print(clean_story_sub_list)

# title, 주소, 조회수, 본문내용 추가한 데이터 프레임
peng_list_update = pd.DataFrame({'title':title_list, 'href':href_list, 'viewcount':clean_viewcount, 'story':clean_story_sub_list})
print(peng_list_update)

['첫 영화 오디션을 본 펭수!라면 먹고 돌아오다?', '범이의 적성을 찾아주기 위해 펭수가 데려간 그곳에서 뜻밖의 재능을 찾게 되는데....============================', '펭수가 헤드셋을 사기당했다!?명탐정이 된 펭수가 벌이는 집요한 조사 끝에 범인은...?============================', 'Pengsoo is Korea TOP creator Penguin!Do you know PENGSOO?독일 Weißt du Pengsoo?프랑스 Vous connaissez Pengsoo?러시아 Вы знаете Пенсу? 아프리카 대륙 هل تعرف بيانج سو؟아르헨티나 ¿Conoces a Pengsoo?중국 你知道朋秀吗?', '펭수의 간을 탐내는 자가 있다????용왕의 사신 해로와 펭수와의 만남 그리고 숨겨진 이야기들펭수판 별주부전', '전원배 매니저(Feat.퇴사자인더하우스) 얼굴에 김이 묻었다.이걸 어떻게 말을 해줘야하나? 말아야 하나?!펭수의 선택에 펭생이 달라진다', '', '신촌에 놀러갔다가 사고쳤는데이거이거 나 판매 소질있는 듯', '(191026) 부산 팬 사인회팬분들과 함께한 그 열광의 현장드디어 공개!', 'K-펭귄 한다 해외진출', '펭수가 화보 모델로서 촬영을 하게 되었다!화보 촬영을 위한 펭수의 눈물(?) 나는 노력!그리고 현직 모델에게 직접 받는 모델 포즈까지!', '힙합펭수의 커버영상이 드. 디. 어. 찾아왔다!월미도에서 촬영했던 미공개 영상과 여러분이 함께 만들어준 이번 신상뮤비 공개챌린지에 참여해준 모든 분들 펭러뷰~♡잊지말자 월미도 우리가 함께한 시간여러분이 저를 위해 함께해주는 시간 모두', "어느 날 갑자기 펭수에게 이상한 행동들이 보인다!? 걱정된 제작진들이 긴급 솔루션 대책 마련에 나섰다!세상에 나쁜 펭귄은 없다!========================= 칠판옆 '꿈을 이루기 위한 7가지 충고 - ' 저자 김효석'=========================", '내일 모레

                                            title                  href  \
0         영화 '천문: 하늘에 묻는다' 오디션 보러 간 펭수(*쿠키영상 있음!)  /watch?v=8qYmLVOlYXI   
1                         [Ep.72] 수업 중에 경운기를 몰아봤다  /watch?v=OD-Y-CxHz4A   
2                          [Ep.71] 내 헤드셋 누가 가져갔어?  /watch?v=rRZtVrh6kgI   
3                            펭수로 세계 대통합은 이미 시작됐다.  /watch?v=RKi98URQ6GM   
4                      [EP.70] 별주부전아닙니다 펭.수.전 입니다  /watch?v=mXUX4V_CYWw   
..                                            ...                   ...   
127  "2m 10cm 전학생  VS 교장선생님?" 펭수, 교장실 급습! [EP.02]  /watch?v=2gg1QKk7fjw   
128          "관종 펭귄, 초등학교 습격?" 펭수, 학교 가다! [EP.01]  /watch?v=M-52pdhp-ao   
129   관종 펭귄 '펭수' EBS 데뷔 D-3!! teaser.2 [자이언트 펭TV]  /watch?v=ErO16Y9sE8Q   
130    'EBS 연습생 펭귄, 방송국 점거??' [자이언트 펭TV] teaser.1  /watch?v=4vkzT47_Vdg   
131              [펭TV] EBS 연습생 펭수 유튜브 데뷔💜 머랭쿠키 먹방  /watch?v=XeH_8w4AWS4   

     viewcount                                              story  
0       950000                 

In [57]:
# 형태소 분석
okt = Okt()

for i in range(len(clean_story_sub_list)):
    print(okt.morphs(clean_story_sub_list[i]))

['첫', '영화', '오디션', '을', '본', '펭수', '!', '라면', '먹고', '돌아오다', '?']
['범', '이의', '적성', '을', '찾아주기', '위해', '펭수', '가', '데려간', '그', '곳', '에서', '뜻밖', '의', '재능', '을', '찾게', '되는데', '....============================']
['펭수', '가', '헤드셋', '을', '사기', '당했다', '!?', '명탐정', '이', '된', '펭수', '가', '벌이는', '집요한', '조사', '끝', '에', '범인', '은', '...?============================']
['Pengsoo', 'is', 'Korea', 'TOP', 'creator', 'Penguin', '!', 'Do', 'you', 'know', 'PENGSOO', '?', '독일', 'Wei', 'ß', 't', 'du', 'Pengsoo', '?', '프랑스', 'Vous', 'connaissez', 'Pengsoo', '?', '러시아', 'Вы', 'знаете', 'Пенсу', '?', '아프리카', '대륙', 'هل', 'تعرف', 'بيانج', 'سو؟', '아르헨티나', '¿', 'Conoces', 'a', 'Pengsoo', '?', '중국', '你知道朋秀吗', '?']
['펭수', '의', '간', '을', '탐내는', '자가', '있다', '????', '용왕', '의', '사신', '해로와', '펭수', '와의', '만남', '그리고', '숨겨진', '이야기', '들', '펭', '수판', '별주부전']
['전원', '배', '매니저', '(', 'Feat', '.', '퇴사', '자', '인더하우스', ')', '얼굴', '에', '김', '이', '묻었다', '.', '이', '걸', '어떻게', '말', '을', '해줘야하나', '?', '말아야', '하나', '?!', '펭수', '의', '선택', '

['방학', '을', '맞이', '하여', '빡빡이', '아저씨', '가', '정성', '으로', '준비', '하고', '펭', '수가', '땀', '한', '바가지', '추가', '한스', '페샬', '운동', '루틴', '비디오', '를', '공개', '합니다', '.#펭수가', '#', '이', '거', '찍다가', '#', '죽', '을', '뻔함']
['본격', '헬린', '이', '양성', '교육', '방송', 'EBS', '!', '세계', '최초', '3', '대', '측정', '하는', '펭귄', ',', '펭수', '!!', '세계', '최초', '펭귄', 'PT', '해주는', '빡저씨', '김', '계란', '!', '지금', '까지', '없었고', '앞', '으로도', '없을', '충격', '과', '공포', '의', '대', '환', '장', '콜라보', '!']
['소듕', '한', '구', '독자', '님', '덜', '을', '위', '한', '홈', '메이드', '굿', '즈', '제작', '공정', '공개', '!']
['*', '눈', '호강', '귀', '호강', '보장', '*', '무려', '기타리스트', '를', '초청', '해서', '무려', '어쿠스틱', '라이브', '로', '불러', '본', '세상', '에서', '제일', '귀여운', '펭수', '라이브', '-', '토이스토리', 'OST', '"', 'You', "'", 've', 'got', 'a', 'friend', 'in', 'me', '"', '우디', '복장', '을', '찰떡', '같이', '소화', '해준', '기타리스트', '정', '소리', '님', '좌표', ':', 'https://music.naver.com/musicianLeagu', '...']
['이번', '생', '말년', '처럼', '보내는', 'EBS', '연습생', '펭수', '와', '침착', '맨', '이말년', '의', '콜라보', '!']
['[', '관전', '포인

['전지영', '스승', '님', '의', '특강', '은', '계속', '된다', '!', '하지만', '펭수', '에겐', '수업', '이', '쉽지만은', '않은데', '....', '선생님', '농구', '가', '....', '아니', '춤', '이', '추고', '싶어요', '..!']
['구', '독자', '를', '더', '끌어', '모으기', '위해', '특별한', '스승', '님', '을', '찾아', '나선', '펭수', ',', '첫', '번째', '스승', '전지영', '스승', '님', '과', '함께', '온라인', "'", '인', '싸', "'", '이', '더', '가', '되기', '위', '한', '날개', '짓', '이', '시작', '된다', '!']
['펭', '-', '하', '!', '펭수', '매니저', '입니다', '.', '이번', '엔', '펭수님', '께서', '요들송', '버스킹', '에', '도전', '했는데요', '.', '잘', '부르는', '건지', '모르', '..', '아', ',', '이', '게', '아니지', '..', '천상', '의', '목소리', '임', '이', '틀림없네요', '^^!']
['(', '공지', ')', '채널', '이름', '은', '팬', 'ㅌ', '..', '아니고', '자이언트', "'", '펭', 'TV', "'", '입니다', '.', '발음', '주의', '.', '깜짝', '인기', '에', '기세', '등등한', '펭수', '!', '친구', '들', '의', '소원', '을', '들어주기', '위해', '교장', '실', '을', '급습', '하고', '맙니', '다', '.', '옆', '에', '있던', '매니저', '는', '펭수', '가', '교장', '선생님', '께', '반모', '신청', '할까', '심장', '이', '오지', '게', '떨렸다는', '후문', '-', '▦', '펭수', '구', '독자', '꿀팁', ':', '딜',

In [58]:
# 명사 분석
for i in range(len(clean_story_sub_list)):
    print(okt.nouns(clean_story_sub_list[i]))

['첫', '영화', '오디션', '펭수', '라면']
['범', '이의', '적성', '위해', '펭수', '곳', '뜻밖', '재능']
['펭수', '헤드셋', '사기', '명탐정', '펭수', '조사', '끝', '범인']
['독일', '프랑스', '러시아', '아프리카', '대륙', '아르헨티나', '중국']
['펭수', '간', '자가', '용왕', '사신', '펭수', '만남', '이야기', '펭', '수판', '별주부전']
['전원', '배', '매니저', '퇴사', '인더하우스', '얼굴', '김', '걸', '말', '하나', '펭수', '선택', '펭생']
[]
['신촌', '사고', '나', '판매', '소질', '듯']
['부산', '팬', '사인회', '팬', '그', '열광', '현장', '공개']
['펭귄', '해외진출']
['펭수', '화보', '모델', '로서', '촬영', '화보', '촬영', '위', '펭수', '눈물', '나', '노력', '현직', '모델', '직접', '모델', '포즈']
['힙합', '펭', '수의', '커버', '영상', '드', '디', '월미도', '촬영', '공개', '영상', '여러분', '준', '이번', '신상', '뮤비', '공개', '린지', '참여', '모든', '펭러뷰', '월미도', '우리', '시간', '여러분', '저', '위해', '시간', '모두']
['날', '갑자기', '펭수', '행동', '걱정', '진', '긴급', '솔루션', '대책', '마련', '세상', '펭귄', '칠판', '옆', '꿈', '위', '가지', '충고', '저자', '김효석']
['내일', '모레', '시험', '여러분', '위해', '이', '영상', '펭펭']
['펭수', '똑', '범', '둘리', '선배', '동창회', '겸', '단풍', '남극', '친구', '동창회', '시간', '잡']
['내레이션', '윤도현', '선배', '비타민', '뿌듯']
['매니저', '적응', '중인',

['피', '드', '백', '천', '사', '펭', '수', '펭수', '프로필', '사진', '댓글', '매니저', '끌', '벚꽃', '화보', '촬영', '쟈']
['계속', '펭수', '남매', '대결', '피', '눈물', '펭수', '앞', '위기', '남매', '운명']
['시작', '펭수', '남매', '크리에이터', '대결', '으뜸', '노력', '펭수', '눈', '만']
['남매', '크리에이터', '체험', '유', '브', '지배', '펭귄', '크리에이터']
['전지영', '스승', '도움', '혜성', '처럼', '전진', '펭수', '앞', '남매', '두번째', '스승', '인기', '크리에이터', '길', '향']
['전지영', '스승', '특강', '계속', '펭수', '수업', '선생님', '농구', '춤']
['독자', '더', '위해', '스승', '나선', '펭수', '첫', '스승', '전지영', '스승', '온라인', '인', '더', '위', '날개', '짓', '시작']
['펭', '펭수', '매니저', '이번', '펭수님', '요들송', '버스킹', '도전', '이', '천상', '목소리', '임']
['공지', '채널', '이름', '팬', '자이언트', '펭', '발음', '주의', '깜짝', '인기', '기세', '펭수', '친구', '소원', '위해', '교장', '실', '급습', '맙니', '옆', '매니저', '펭수', '교장', '선생님', '반모', '신청', '심장', '오지', '후문', '펭수', '독자', '꿀팁', '딜', '때', '일단', '것']
['학년', '교실', '자이언트', '펭귄', '포스트', '뽀로로', '위해', '남극', '헤엄', '온', '펭수', '산초', '초딩', '덕심', '수', '의', '연습생', '펭수', '데뷔', '에피소드']
['우리', '펭수', '꽃길', '펭수', '데뷔', '실화', '자이언트', '펭', '첫', '방송']
[

In [59]:
# 구(Phrase) 분석
for i in range(len(clean_story_sub_list)):
    print(okt.phrases(clean_story_sub_list[i]))

['첫 영화', '첫 영화 오디션', '펭수', '라면', '영화', '오디션']
['범이의', '범이의 적성', '위해', '위해 펭수', '그곳', '뜻밖', '뜻밖의 재능', '이의', '적성', '펭수', '재능']
['펭수', '헤드셋', '사기', '명탐정', '집요한 조사', '집요한 조사 끝', '범인', '조사']
['Pengsoo', 'Pengsoo is', 'Pengsoo is Korea', 'Pengsoo is Korea TOP', 'Pengsoo is Korea TOP creator', 'Do', 'Do you', 'Do you know', 'Do you know PENGSOO', '독일', '독일 Wei', 't du', 't du Pengsoo', '프랑스', '프랑스 Vous', '프랑스 Vous connaissez', '프랑스 Vous connaissez Pengsoo', '러시아', '아프리카', '아프리카 대륙', '아르헨티나', 'Conoces', 'Conoces a', 'Conoces a Pengsoo', '중국', 'is', 'Korea', 'TOP', 'creator', 'Penguin', 'you', 'know', 'PENGSOO', 'Wei', 'du', 'Vous', 'connaissez', '대륙']
['펭수', '펭수의 간', '자가', '용왕', '용왕의 사신', '만남', '이야기들펭수판', '이야기들펭수판 별주부전', '사신', '이야기들', '수판', '별주부전']
['전원배', '전원배 매니저', 'Feat', '퇴사자인더하우스', '얼굴', '이걸', '하나', '펭수', '펭수의 선택', '펭생', '전원', '매니저', '퇴사자', '인더하우스', '선택']
[]
['신촌', '사고', '나 판매', '나 판매 소질', '나 판매 소질있는 듯', '판매', '소질']
['191026', '부산', '부산 팬', '부산 팬 사인회팬분들', '부산 팬 사인회팬분들과 함께한 그', '부산 팬 사인회팬분

['지난', '지난 고양예고편', '미처', '펭수x고양예고', '펭수x고양예고 마피아게임', '펭수x고양예고 마피아게임 분량', '펭수x고양예고 마피아게임 분량 공개', '시민vs', '마피아', '마피아 숨', '추리', '추리 대결', '펭수', '바닥', '뒹구는데', '과연', '과연 무슨', '과연 무슨 일이', '고양예고편', '고양예고', '마피아게임', '분량', '공개', '시민', 'vs', '대결', '무슨', '일이']
['훈민정음게임', '탁구공굴리기야구', '9회', '2아웃', '게임대회', '2라운드', '훈민정음', '게임', '탁구', '공굴리기', '야구', '아웃', '대회', '라운드']
['바보게임', '베스킨라빈스31너희', '이 게임', '이 게임 알', '펭수', '바보', '게임', '베스킨라빈스', '31', '너희']
['펭수', '고양예고', '펭수 진로', '탐색', '친구들', '정보', '가지', '앨렌쇼', '펭수쇼', '직접', '직접 고양예고', '직접 고양예고의 각', '직접 고양예고의 각 과', '직접 고양예고의 각 과 선배님', '직접 고양예고의 각 과 선배님들을 모시', '구독자들', '궁금해하는 것들', '대해', '이야기', '예술계', '예술계 진학', '있는 친구들', '주목', '진로', '앨렌', '선배님', '모시', '것들', '진학']
['펭수', '연기', '문예창작', '무용', '적성', '적성 검사', '검사']
['인성', '펭성', '펭수', '바이브', '바이브 뿜뿜하', '음악', '미술', '미술과 실기시험', '미술과 실기시험 도전', '뿜뿜하', '실기', '시험', '도전']
['0r', 'girl', '마법소녀물', '펭수', '펭수의 직업체험', '성우편', '성우편 두', '이야기', '직업', '체험']
['성우님', '전적', '펭수의', '펭수의 직업체험', '펭수의 직업체험 1', 'EBS', 'EBS 성우이자', 'EBS 성우이자 유명',

In [60]:
# 품사 태깅
for i in range(len(clean_story_sub_list)):
    print(okt.pos(clean_story_sub_list[i]))

[('첫', 'Noun'), ('영화', 'Noun'), ('오디션', 'Noun'), ('을', 'Josa'), ('본', 'Verb'), ('펭수', 'Noun'), ('!', 'Punctuation'), ('라면', 'Noun'), ('먹고', 'Verb'), ('돌아오다', 'Verb'), ('?', 'Punctuation')]
[('범', 'Noun'), ('이의', 'Noun'), ('적성', 'Noun'), ('을', 'Josa'), ('찾아주기', 'Verb'), ('위해', 'Noun'), ('펭수', 'Noun'), ('가', 'Josa'), ('데려간', 'Verb'), ('그', 'Determiner'), ('곳', 'Noun'), ('에서', 'Josa'), ('뜻밖', 'Noun'), ('의', 'Josa'), ('재능', 'Noun'), ('을', 'Josa'), ('찾게', 'Verb'), ('되는데', 'Verb'), ('....============================', 'Punctuation')]
[('펭수', 'Noun'), ('가', 'Josa'), ('헤드셋', 'Noun'), ('을', 'Josa'), ('사기', 'Noun'), ('당했다', 'Adjective'), ('!?', 'Punctuation'), ('명탐정', 'Noun'), ('이', 'Josa'), ('된', 'Verb'), ('펭수', 'Noun'), ('가', 'Josa'), ('벌이는', 'Verb'), ('집요한', 'Adjective'), ('조사', 'Noun'), ('끝', 'Noun'), ('에', 'Josa'), ('범인', 'Noun'), ('은', 'Josa'), ('...?============================', 'Punctuation')]
[('Pengsoo', 'Alpha'), ('is', 'Alpha'), ('Korea', 'Alpha'), ('TOP', 'Alpha'), ('creator', 'Alp

[('이', 'Noun'), ('영광', 'Noun'), ('을', 'Josa'), ('구', 'Modifier'), ('독자', 'Noun'), ('여러분', 'Noun'), ('들', 'Suffix'), ('께', 'Josa'), ('돌립니당', 'Verb')]
[('펭수', 'Noun'), ('가', 'Josa'), ('펭귄', 'Noun'), ('이', 'Josa'), ('아니다', 'Adjective'), ('?!?!?', 'Punctuation'), ('펭수', 'Noun'), ('를', 'Josa'), ('둘러싼', 'Verb'), ('정체', 'Noun'), ('에', 'Josa'), ('대한', 'Noun'), ('의혹', 'Noun'), ('낱낱이', 'Noun'), ('파헤쳐', 'Verb'), ('보겠습니다', 'Verb'), ('펭펭', 'Noun'), ('!!', 'Punctuation')]
[('펭수', 'Noun'), ('당근', 'Noun'), ('10', 'Number'), ('살', 'Noun'), ('맞거든', 'Verb'), ('용', 'Noun'), ('?(', 'Punctuation'), ('당근', 'Noun'), ('을', 'Josa'), ('흔든다', 'Verb'), (')', 'Punctuation')]
[('목요일', 'Noun'), ('20시', 'Number'), ('30분', 'Number'), ('전', 'Modifier'), ('주재', 'Noun'), ('방금', 'Noun'), ('요일', 'Noun'), ('20시', 'Number'), ('30분', 'Number'), ('본방', 'Noun'), ('연속', 'Noun'), ('2', 'Number'), ('편', 'Noun'), ('일요일', 'Noun'), ('12시', 'Number'), ('30분', 'Number'), ('전', 'Modifier'), ('주재', 'Noun'), ('방', 'Noun'), (',', 'Punctuati

[('"', 'Punctuation'), ('반', 'Noun'), ('에서', 'Josa'), ('몇', 'Noun'), ('등', 'Noun'), ('이니', 'Josa'), ('?"', 'Punctuation'), ('잔소리', 'Noun'), ('하면', 'Verb'), ('벌금', 'Noun'), ('이', 'Josa'), ('xxxxxxx', 'Alpha'), ('원', 'Noun'), ('이라고', 'Josa'), ('?', 'Punctuation'), ('명절', 'Noun'), ('잔소리', 'Noun'), ('유형', 'Noun'), ('+', 'Punctuation'), ('벌금', 'Noun'), ('뿌셔', 'Noun'), ('!', 'Punctuation')]
[('공포', 'Noun'), ('의', 'Josa'), ('효도', 'Noun'), ('펭귄', 'Noun'), ('특집', 'Noun'), ('!', 'Punctuation'), ('어르신', 'Noun'), ('분들', 'Suffix'), ('괜찮아요', 'Adjective'), ('..?', 'Punctuation')]
[('친구', 'Noun'), ('들', 'Suffix'), ('펭하', 'Noun'), ('!', 'Punctuation'), ('이번', 'Noun'), ('에', 'Josa'), ('펭수', 'Noun'), ('EBS', 'Alpha'), ('로비', 'Noun'), ('에서', 'Josa'), ('컷팅식', 'Noun'), ('했어펭수', 'Noun'), ('는', 'Josa'), ('실물', 'Noun'), ('이', 'Josa'), ('낫', 'Noun'), ('지만', 'Josa'), ('한번', 'Noun'), ('보러', 'Verb'), ('놀러와', 'Noun')]
[('펭수', 'Noun'), ('와', 'Josa'), ('구', 'Modifier'), ('독자', 'Noun'), ('의리', 'Noun'), ('테스트', 'Noun')

[('방학', 'Noun'), ('을', 'Josa'), ('맞이', 'Noun'), ('하여', 'Verb'), ('빡빡이', 'Noun'), ('아저씨', 'Noun'), ('가', 'Josa'), ('정성', 'Noun'), ('으로', 'Josa'), ('준비', 'Noun'), ('하고', 'Verb'), ('펭', 'Noun'), ('수가', 'Noun'), ('땀', 'Noun'), ('한', 'Verb'), ('바가지', 'Noun'), ('추가', 'Noun'), ('한스', 'Noun'), ('페샬', 'Noun'), ('운동', 'Noun'), ('루틴', 'Noun'), ('비디오', 'Noun'), ('를', 'Josa'), ('공개', 'Noun'), ('합니다', 'Verb'), ('.#펭수가', 'Hashtag'), ('#', 'Punctuation'), ('이', 'Determiner'), ('거', 'Noun'), ('찍다가', 'Verb'), ('#', 'Punctuation'), ('죽', 'Noun'), ('을', 'Josa'), ('뻔함', 'Adjective')]
[('본격', 'Noun'), ('헬린', 'Noun'), ('이', 'Josa'), ('양성', 'Noun'), ('교육', 'Noun'), ('방송', 'Noun'), ('EBS', 'Alpha'), ('!', 'Punctuation'), ('세계', 'Noun'), ('최초', 'Noun'), ('3', 'Number'), ('대', 'Verb'), ('측정', 'Noun'), ('하는', 'Verb'), ('펭귄', 'Noun'), (',', 'Punctuation'), ('펭수', 'Noun'), ('!!', 'Punctuation'), ('세계', 'Noun'), ('최초', 'Noun'), ('펭귄', 'Noun'), ('PT', 'Alpha'), ('해주는', 'Verb'), ('빡저씨', 'Noun'), ('김', 'Noun'), ('계란', 

[('아이돌', 'Noun'), ('이', 'Josa'), ('되고싶어', 'Verb'), ('?', 'Punctuation'), ('연습생', 'Noun'), ('이', 'Josa'), ('궁금해', 'Adjective'), ('?', 'Punctuation'), ('유명', 'Noun'), ('기획사', 'Noun'), ('연습생', 'Noun'), ('들', 'Suffix'), ('과', 'Josa'), ('함께', 'Adverb'), ('파헤치는', 'Verb'), ('연습생', 'Noun'), ('A', 'Alpha'), ('to', 'Alpha'), ('Z', 'Alpha'), ('!!', 'Punctuation'), ('펭수', 'Noun'), ('의', 'Josa'), ('연습생', 'Noun'), ('회담', 'Noun'), ('지금', 'Noun'), ('시작', 'Noun'), ('합니다', 'Verb')]
[('졸사', 'Verb'), ('시즌', 'Noun'), ('이', 'Josa'), ('다가왔다', 'Verb'), ('!', 'Punctuation'), ('눈', 'Noun'), ('감는', 'Verb'), ('애', 'Noun'), (',', 'Punctuation'), ('화장', 'Noun'), ('진한', 'Adjective'), ('애', 'Noun'), ('..', 'Punctuation'), ('꼭', 'Noun'), ('있는', 'Adjective'), ('거', 'Noun'), ('RGRG', 'Alpha'), ('??', 'Punctuation'), ('틱톡', 'Noun'), ('스타', 'Noun'), ('김다윤', 'Noun'), ('과', 'Josa'), ('함께한', 'Verb'), ('펭수', 'Noun'), ('의', 'Josa'), ("'", 'Punctuation'), ('졸업사진', 'Noun'), ('공감', 'Noun'), ("'", 'Punctuation'), ('이번', 'Noun'), (

[('펭수', 'Noun'), ('의', 'Josa'), ('홈', 'Noun'), ('스윗', 'Noun'), ('홈', 'Noun'), ('은', 'Josa'), ('지하', 'Noun'), ('소품', 'Noun'), ('실', 'Noun'), ('?!', 'Punctuation'), ('EBS', 'Alpha'), ('직원', 'Noun'), ('복지', 'Noun'), ('실화', 'Noun'), ('냐', 'Josa'), ('...', 'Punctuation'), ('침실', 'Noun'), ('공개', 'Noun'), ('부터', 'Josa'), ('하드', 'Noun'), ('트레이닝', 'Noun'), ('과', 'Josa'), ('회의', 'Noun'), (',', 'Punctuation'), ('편집', 'Noun'), ('까지', 'Josa'), ('!!', 'Punctuation'), ('스페셜', 'Noun'), ('게스트', 'Noun'), ('와', 'Josa'), ('함께', 'Adverb'), ('하는', 'Verb'), ('펭수', 'Noun'), ('의', 'Josa'), ('은밀', 'Noun'), ('한', 'Josa'), ('사생활', 'Noun'), ('대', 'Modifier'), ('공개', 'Noun'), ('!!', 'Punctuation'), ('기대하', 'Adjective'), ('show', 'Alpha'), ('show', 'Alpha'), ('show', 'Alpha'), ('~', 'Punctuation')]
[('(', 'Punctuation'), ('눈물', 'Noun'), ('주의', 'Noun'), (')', 'Punctuation'), ('~', 'Punctuation'), ('뽀로로', 'Noun'), ('VS', 'Alpha'), ('펭수', 'Noun'), ('-', 'Punctuation'), ('엔드게임', 'Noun'), ('~', 'Punctuation'), ('진짜', 'No